#### Load libraries

In [1]:
# standard libraries
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import tree
from sklearn.metrics import roc_auc_score


# from our code-base
import pac_explanation
from data.objects import zoo, iris, adult, anchor_dataset_wrap
from pac_explanation import example_queries
from pac_explanation.blackbox import *
from pac_explanation.learner import *
from pac_explanation.query import *
from pac_explanation.sygus_if import *
from pac_explanation.teacher import *

### Load dataset

Extra information that we need are following.

(1) `attribute_names` : feature names of the dataset,

(2) `attribute_type`: a map between an attribute and its data type; we consider either "Bool", "Categorical" or "Real" (numerical) attributes.

(3) `real_attribute_domain_info` (optional): for real-valued attributes, we construct a map where the key is the attribute and the value is a tuple (max_value, min_value) for that attribute. In case `real_attribute_domain_info` is incomplete or not provided, we assume that the attribute values are within \[0,1\]

(4) `categorical_attribute_domain_info` (optional): for categorical attributes, we similarly construct a map where the key is the attribute and the value is the possible categories that the attribute can take. If `categorical_attribute_domain_info` is not provided or incomplete, we assumt that the attribute to be Boolean and thus the attribute can be either 0 or 1 

For example, see the [adult dataset](data/objects/adult.py)

Since scikit-learn classifiers do not allow us to specify categorical features, we apply one-hot encoding on categorical features due to practical concerns. However, note that our SyGuS-based classifier does not have such a restriction.





In [2]:
# choose one dataset
dataset = ['zoo', 'adult', 'iris', "anchor_adult"][3]

df = None

if(dataset == "zoo"):
    dataObj = zoo.Zoo()
    df = dataObj.get_df()
    # fix target class because zoo dataset has 7 classes while we support binary class datasets in this implementation
    target_class = [4] 
    _temp = {}
    for i in range(1, len(df[dataObj.target].unique())+1):
        if(i in target_class):
            _temp[i] = 1
        else:
            _temp[i] = 0
    df[dataObj.target] = df[dataObj.target].map(_temp)
elif(dataset == "adult"):
    dataObj = adult.Adult() 
    df = dataObj.get_df()
elif(dataset == "iris"):
    dataObj = iris.Iris()
    df = dataObj.get_df()
elif(dataset == "anchor_adult"):
    dataObj = anchor_dataset_wrap.Anchor(dataset_name="adult")
    df = dataObj.get_df()


df.head()

Calling one hot encoder on ['Workclass', 'Education', 'MaritalStatus', 'Occupation', 'Relationship', 'Race', 'CapitalGain', 'CapitalLoss', 'Country', 'Age', 'Hoursperweek']


,Sex,target,Workclass_0.0,Workclass_1.0,Workclass_2.0,Workclass_3.0,Workclass_4.0,Workclass_5.0,Workclass_6.0,Workclass_7.0,...,Country_39.0,Country_40.0,Country_41.0,Age_0.0,Age_1.0,Age_2.0,Age_3.0,Hoursperweek_0.0,Hoursperweek_1.0,Hoursperweek_2.0
0,1.0,1,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,1,0
1,0.0,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,0
2,0.0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,1,0,0
3,1.0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,1,0,0
4,1.0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,1,0,0


#### More information about the dataset

In [3]:
print("Attributes:", dataObj.attributes, "\n")
print("Attributes type (without class):", dataObj.attribute_type, "\n")
print("Domain information of real-valued attributes:\n", dataObj.real_attribute_domain_info, "\n")
print("Domain information of categorical attributes:\n", dataObj.categorical_attribute_domain_info, "\n")
print("Boolean attributes:", dataObj.Boolean_attributes)

Attributes: ['Sex', 'Workclass_0.0', 'Workclass_1.0', 'Workclass_2.0', 'Workclass_3.0', 'Workclass_4.0', 'Workclass_5.0', 'Workclass_6.0', 'Workclass_7.0', 'Workclass_8.0', 'Education_0.0', 'Education_1.0', 'Education_2.0', 'Education_3.0', 'Education_4.0', 'Education_5.0', 'Education_6.0', 'Education_7.0', 'Education_8.0', 'Education_9.0', 'Education_10.0', 'Education_11.0', 'Education_12.0', 'Education_13.0', 'Education_14.0', 'Education_15.0', 'MaritalStatus_0.0', 'MaritalStatus_1.0', 'MaritalStatus_2.0', 'MaritalStatus_3.0', 'MaritalStatus_4.0', 'MaritalStatus_5.0', 'MaritalStatus_6.0', 'Occupation_0.0', 'Occupation_1.0', 'Occupation_2.0', 'Occupation_3.0', 'Occupation_4.0', 'Occupation_5.0', 'Occupation_6.0', 'Occupation_7.0', 'Occupation_8.0', 'Occupation_9.0', 'Occupation_10.0', 'Occupation_11.0', 'Occupation_12.0', 'Occupation_13.0', 'Occupation_14.0', 'Relationship_0.0', 'Relationship_1.0', 'Relationship_2.0', 'Relationship_3.0', 'Relationship_4.0', 'Relationship_5.0', 'Race_0

#### Dataset preparation for training

In [4]:
# declaration of classifier, X and y
X = df.drop([dataObj.target], axis=1)
y = df[dataObj.target]

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle = True, random_state=2) # 70% training and 30% test

### Train a black-box model

In [5]:
# Preload/store trained model

select_blackbox = ['decision tree','neural network', 'random forest'][0]

model_name = None
if(select_blackbox == 'decision tree'):
    model_name = 'data/model/dt_' + dataset + '.pkl'
elif(select_blackbox == "random forest"):
    model_name = 'data/model/rf_' + dataset + '.pkl'
elif(select_blackbox == "neural network"):
    model_name = 'data/model/nn_' + dataset + '.pkl'

else:
    raise ValueError("Black box not defined")



if(not os.path.isfile(model_name)):
    clf = None
    if(select_blackbox == 'decision tree'):
        param_grid = {'max_depth': np.arange(3, 10)}
        grid_tree = GridSearchCV(tree.DecisionTreeClassifier(random_state=0), param_grid)
        grid_tree.fit(X_train, y_train)
        tree_preds = grid_tree.predict_proba(X_test)[:, 1]
        tree_performance = roc_auc_score(y_test, tree_preds)
        clf = grid_tree.best_estimator_
    elif(select_blackbox == "random forest"):
        clf = RandomForestClassifier(n_estimators=100)
        clf.fit(X_train,y_train)

    elif(select_blackbox == "neural network"):
        clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train) 
        clf.fit(X_train,y_train)
        
    else:
        raise ValueError("Black box not defined")

    # store the classifier
    with open(model_name, 'wb') as fid:
        pickle.dump(clf, fid)    

else:
    print("Loding model")
    with open(model_name, 'rb') as fid:
        clf = pickle.load(fid)

    
# For visualization
if(select_blackbox == "decision tree"):
    print(pac_explanation.utils.tree_to_code(clf,X_train.columns.tolist()))



       else:
                                return 0
                else:
                    if Education_2.0 <= 0.5:
                        if Education_9.0 <= 0.5:
                            if Race_1.0 <= 0.5:
                                if Education_7.0 <= 0.5:
                                    if Education_14.0 <= 0.5:
                                        return 0
                                    else:
                                        return 0
                                else:
                                    if Hoursperweek_2.0 <= 0.5:
                                        return 0
                                    else:
                                        return 1
                            else:
                                if Hoursperweek_1.0 <= 0.5:
                                    if Occupation_1.0 <= 0.5:
                                        return 0
                                    else:
                                  

## Specify a target sample (i.e., query with a distance parameter)
In this experiment, we have considered two types of queries: 
    
    (1) target sample,
    (2) a logical specification of the sample space viewed as a region    
    
for which, we learn a local explanation of the black-box model

In [10]:
select_query = ['decision tree', 'specific input'][1]

# X and y will be used as arguments for learner as initial examples. While this is not a strict requirement, we make sure that the learner is atleast correct with respect to the given sample (when select_query = specific_input).

X = None
y = None 

# Our PAC explainer considers a query object. For either type of queries, we define a query_class
query_class = None
if(select_query == "decision tree"):
    # We define query specilized for decision tree
    query_class = example_queries.DecisionTree(features=X_train.columns.tolist(), halfspace=example_queries.ExampleQueries().queries[0])
    X = []
    y = []
elif(select_query == "specific input"):  
    # An input from the training set
    specific_input = X_test.iloc[0].tolist()

    # A threshold is provided for distance based query. The range of this threshold is not properly checked. Roughly, all neighboring samples with distance less than the threshold are considered inside the local region
    query_class = example_queries.DistanceQuery(specific_input=specific_input, threshold=utils.learn_threshold(specific_input,X_test.values, quantile_val=.2), features = X_train.columns.tolist())
    X = [specific_input]
    y = [clf.predict([specific_input])[0]]
    print(query_class)
    print("Label of this sample by the black box predictor:", y)
    
else:
    raise ValueError(select_query +" is not a defined query.")

Learned threshold: 0.5
Query is a specific input
- threshold: 0.5
- specific_input: [1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]
- detailed_input: [('Sex', 1.0), ('Workclass_0.0', 0.0), ('Workclass_1.0', 0.0), ('Workclass_2.0', 0.0), ('Workclass_3.0', 0.0), ('Workclass_4.0', 1.0), ('Workclass_5.0', 0.0), ('Workclass_6.0', 0.0), ('Workclass_7.0', 0.0), ('Workclass_8.0', 0.0), ('Education_0.0', 0.0), ('Education_1.0', 0.0), ('Education_2.0', 0.0), ('Education_3.0', 0.0), ('Edu

## Generate a PAC explanation with SyGuS
We define three entities here.

    (1) blackbox,
    (2) learner, and
    (3) query

Our task is to define a `teacher` who performs the task of learning an explanation of the `blackbox` given the `query` by sending necessary information to the `learner`.

In [11]:
# define the black-box as a scikit-learn object (trained)
blackbox = None
if(select_blackbox == 'decision tree' or select_blackbox == "random forest" or select_blackbox == "neural network"):
    blackbox = BlackBox(clf, clf.predict)
else:
    raise ValueError("Black box not defined")

# define the learner as a SyGuS object
sygus = SyGuS_IF(rule_type="CNF", feature_names=dataObj.attributes, feature_data_type=dataObj.attribute_type, function_return_type= "Bool", real_attribute_domain_info=dataObj.real_attribute_domain_info, categorical_attribute_domain_info=dataObj.categorical_attribute_domain_info, verbose=False, syntactic_grammar=True, workdir="temp")

learner = Learner(model = sygus, prediction_function = sygus.predict_z3, train_function = sygus.fit, X = X, y=y)

# define the query as specified above
query = Query(model = None, prediction_function = query_class.predict_function_query)


### Teacher

In [12]:
# Now let the teacher complete its task
teacher = Teacher(max_iterations=100000, epsilon=0.001, delta=0.001, timeout=10)

# Teacher object needs an access to a random sample generator for the verification task. We call a specilized random generator that utilizes the knowledge of the training dataset, for example, the attribute type of each attribute.

teach_start = time.time()
learner, flag = teacher.teach(blackbox = blackbox, learner = learner, query = query, random_example_generator = utils.random_generator, params_generator = (X_train, dataObj.attribute_type), verbose=False)
teach_end = time.time()

# print("Explanation for =>")
# print(query_class)
print("\nLearned explanation =>")
print(learner.model._function_snippet)
print("\n-explanation size:", learner.model.get_formula_size(verbose=False))
print("-is explanation learning complete?", flag)
print("-it took", teach_end - teach_start, "seconds")
print("-learner time:", teacher.time_learner)
print("-verifier time:", teacher.time_verifier)
print('-random words checked', teacher.verifier.number_of_examples_checked)
print("-random words filtered by querys:", teacher.verifier.filtered_by_query)
print("-total counterexamples:", len(learner.y))
print("-percentage of positive counterexamples for the learner:", np.array(learner.y).mean())



Learned explanation =>
None
Function snippet is None

-explanation size: 0
-is explanation learning complete? True
-it took 1.3902878761291504 seconds
-learner time: 0
-verifier time: 1.3900480270385742
-random words checked 0
-random words filtered by querys: 7601
-total counterexamples: 1
-percentage of positive counterexamples for the learner: 1.0


#### Evaluation of the explanation

In [13]:
# If a test sample is not within the query, we do not report explanation error, otherwise we report error when the learner and the blackbox have different classification for that sample

acc = None
try:
    cnt = 0
    learner_verdicts = learner.classify_examples(X_test.values.tolist())
    blackbox_verdicts = blackbox.classify_examples(X_test.values.tolist())
    for i in range(len(X_test.values.tolist())):

        blackbox_verdict = blackbox_verdicts[i]
        learner_verdict = learner_verdicts[i]
        query_verdict = query.classify_example(X_test.values.tolist()[i])
        if(not query_verdict):
            cnt += 1
            continue
        if(learner_verdict == blackbox_verdict):
            cnt += 1
    acc = cnt/len(y_test)
except:
    cnt = None
    acc = None

print("correct: ", cnt, "out of ", len(y_test), "examples. Percentage: ", acc)

correct:  None out of  1412 examples. Percentage:  None


### Anchor explanation

In [14]:
import anchor.utils
from anchor import anchor_tabular

# compatible for Anchor input
anchor_categorical_names = {}
cnt = 0
for attribute in dataObj.attributes:
    assert dataObj.attribute_type[attribute] == "Bool"
    anchor_categorical_names[cnt] = ["0","1"]
    cnt += 1


explainer = anchor_tabular.AnchorTabularExplainer(
    [0,1],
    dataObj.attributes,
    np.array(X_train),
    anchor_categorical_names
)

# target sample
idx = 0
np.random.seed(1)
print('Prediction: ', explainer.class_names[clf.predict(np.array(X_test)[idx].reshape(1, -1))[0]])
exp = explainer.explain_instance(np.array(X_test)[idx], clf.predict, threshold=0.95)

print('Anchor: %s' % (' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Prediction:  1
Anchor: CapitalLoss_2.0 = 1 AND CapitalLoss_0.0 = 0 AND Occupation_4.0 = 1
Precision: 0.97
Coverage: 0.01
